In [1]:
import scrapy
import numpy as np
from scrapy.crawler import CrawlerProcess
import re
import json

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('span.Estilo67::text').extract_first()
        nombre = limpiaText(nombre) # cambié esto
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('td.textoNegro::text').extract()[5]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['CURP']=curp # Agregué esto para 
        dipus.get(curp,{})['Nombre']=nombre # cambié aquí
        dipus.get(curp,{})['TipoElección']=response.css('td.textoNegro::text').extract()[0]
        dipus.get(curp,{})['Entidad']=response.css('td.textoNegro::text').extract()[1]
        parti=response.xpath('//tr/td/table//td//img/@src').extract()[1]
        dipus.get(curp,{})['Partido']=obtienePartido(parti)
        distri1 =response.css('td.textoNegro::text').extract()[2]
        distri2 =response.css('td.textoNegro::text').extract()[4]  # PARTE RARA
        dipus.get(curp,{})['Distrito'] = distri1 +""+ distri2
        dipus.get(curp,{})['Correo']=response.css('a.linkNegroSin::text').extract()[0]
        dipus.get(curp,{})['Onomástico']=limpiaText(cumple)
        supl=response.css('span.Estilo67::text').extract()[6]
        dipus.get(curp,{})['Suplente']=limpiaText(supl)
        dipus.get(curp,{})['Comisiones']=response.xpath('//a[contains(@href, "integrantes_de_comision")]/text()').getall()
        dipus.get(curp,{})['Último grado de estudios']=response.xpath('//td[@class="textoNegro"]/text()').extract_first()
        link_asis=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[2]
        link_votaciones=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[3]
        #yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
        #                     callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            if(votacion in por_votaciones.keys()):
                por_votaciones[votacion][curp]=posi
            else:
                    por_votaciones[votacion] = {"ID_voto":limpiaVotacion(votacion),"Nombre_Votación":votacion, curp:posi}
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    '''
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    '''
    curp = nombre.replace('.', '')
    curp = curp.replace(' ', '')
    curp = curp.upper()
    curp = curp.replace('Á', 'A')
    curp = curp.replace('É', 'E')
    curp = curp.replace('Í', 'I')
    curp = curp.replace('Ó', 'O')
    curp = curp.replace('Ú', 'U')
    curp = curp.replace('Ñ', 'N')
    return curp

In [4]:
def limpiaText(nom):
    resp = re.sub('\s+',' ',nom)
    resp = resp.replace('Dip. ', '')
    resp = resp.replace('(LICENCIA)', '')
    resp = resp.replace('Suplente: ', '')
    resp = resp.replace('(DECESO)', '')
    resp = resp.replace('(Deceso)', '')
    resp = resp.replace('  ', ' ')
    if(resp!="" and resp[-1]==" "):
        resp = resp[0:len(resp)-1]
    if(resp!="" and resp[0]==" "):
        resp = resp[1:]
    return resp

In [5]:
def limpiaVotacion(nombreVot):
    nombreVot = nombreVot.replace('(', '')
    nombreVot = nombreVot.replace(')', '')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace('DECRETO', '')
    nombreVot = nombreVot.replace(' QUE ', ' ')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace(' DE ', ' ')
    nombreVot = nombreVot.replace(' A ', ' ')
    nombreVot = nombreVot.replace(' LA ', ' ')
    nombreVot = nombreVot.replace(' EL ', ' ')
    nombreVot = nombreVot.replace(' POR ', ' ')
    nombreVot = nombreVot.replace(' PARA ', ' ')
    nombreVot = nombreVot.replace(' LOS ', ' ')
    nombreVot = nombreVot.replace(' LAS ', ' ')
    nombreVot = nombreVot.replace(' Y ', ' ')
    nombreVot = nombreVot.replace(' E ', ' ')
    nombreVot = nombreVot.replace(' CON ', ' ')
    nombreVot = nombreVot.replace(' LEY ', ' ')
    nombreVot = nombreVot.replace(' EN ', ' ')
    nombreVot = nombreVot.replace(' LO ', ' ')
    nombreVot = nombreVot.replace(' DEL ', ' ')
    nombreVot = nombreVot.replace('.', ' ')
    nombreVot = nombreVot.replace(';', ' ')
    nombreVot = nombreVot.replace(',', ' ')
    nombreVot = nombreVot.replace('  ', ' ')
    nombreFin=""
    for s in nombreVot.split():
        nombreFin+=s[0]
    return nombreFin
    

In [6]:
def obtienePartido(imagen):
    res=""
    if(imagen=='images/pan.png'):
        res = "PAN"
    elif(imagen=='images/logpt.jpg'):
        res = "PT"
    elif(imagen=='images/LogoMorena.jpg'):
        res = "Morena"
    elif(imagen=='images/encuentro.png'):
        res = "Encuentro Social"
    elif(imagen=='images/logo_movimiento_ciudadano.png'):
        res = "Movimiento Ciudadano"
    elif(imagen=='images/pri01.png'):
        res = "PRI"
    elif(imagen=='images/logvrd.jpg'):
        res = "Partido Verde"
    elif(imagen=='images/prd01.png'):
        res = "PRD"
    elif(imagen=='images/logo_SP.jpg'):
        res = "Sin Partido"
    elif(imagen=="images/panal.gif"):
        res = "Nueva Alianza"
    elif(imagen=="images/independiente.png"):
        res = "Independiente"
    else:
         res = imagen
    return res
                                    

In [7]:
dipus={}
curp = "h"
urlDipu = "http://sitllxii.diputados.gob.mx/"
urlCurr = "http://sitllxii.diputados.gob.mx/curricula.php?dipt="

por_votaciones = {}

terminaciones = []
for i in range(0,5):
    for j in range(0,10):
        for k in range(0,10):
            terminaciones.append(str(i)+str(j)+str(k))
terminaciones.append(str(500))
terminaciones.pop(0)


# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-04-09 15:53:04 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-04-09 15:53:04 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-04-09 15:53:04 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-04-09 15:53:04 [scrapy.crawler] INFO: Overridden settings:
{}
2021-04-09 15:53:05 [scrapy.extensions.telnet] INFO: Telnet Password: 5a56009dc0ef8e3b
2021-04-09 15:53:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-04-09 15:53:05 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-04-09 15:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=012> (referer: http://sitllxii.diputados.gob.mx/curricula.php?dipt=012)
2021-04-09 15:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=016> (referer: http://sitllxii.diputados.gob.mx/curricula.php?dipt=016)
2021-04-09 15:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=020> (referer: http://sitllxii.diputados.gob.mx/curricula.php?dipt=020)
2021-04-09 15:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=014> (referer: http://sitllxii.diputados.gob.mx/curricula.php?dipt=014)
2021-04-09 15:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonpl

2021-04-09 15:53:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=22&pert=15> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=022)
2021-04-09 15:53:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=22&pert=7> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=022)
2021-04-09 15:53:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=22&pert=5> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=022)
2021-04-09 15:53:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=22&pert=14> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=022)
2021-04-09 15:53:18 [scrapy.core.engin

2021-04-09 15:53:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=17&pert=5> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=017)
2021-04-09 15:53:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=17&pert=14> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=017)
2021-04-09 15:53:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=17&pert=13> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=017)
2021-04-09 15:53:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=17&pert=1> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=017)
2021-04-09 15:53:28 [scrapy.core.engin

2021-04-09 15:53:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=18&pert=14> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=018)
2021-04-09 15:53:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=18&pert=13> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=018)
2021-04-09 15:53:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=18&pert=1> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=018)
2021-04-09 15:53:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=14&pert=9> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=014)
2021-04-09 15:53:45 [scrapy.core.engin

...

2021-04-09 16:35:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=478&pert=14> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=478)
2021-04-09 16:35:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=478&pert=15> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=478)
2021-04-09 16:35:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=478&pert=13> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=478)
2021-04-09 16:35:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=478&pert=1> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=478)
2021-04-09 16:35:20 [scrapy.core.

2021-04-09 16:35:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=492&pert=9> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=492)
2021-04-09 16:35:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=492&pert=18> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=492)
2021-04-09 16:35:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=492&pert=17> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=492)
2021-04-09 16:35:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=492&pert=16> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=492)
2021-04-09 16:35:36 [scrapy.core.

2021-04-09 16:35:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=476&pert=1> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=476)
2021-04-09 16:35:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=476&pert=3> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=476)
2021-04-09 16:35:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=475&pert=3> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=475)
2021-04-09 16:35:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=475&pert=1> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=475)
2021-04-09 16:35:54 [scrapy.core.eng

2021-04-09 16:36:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=499&pert=17> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=499)
2021-04-09 16:36:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=499&pert=16> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=499)
2021-04-09 16:36:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=499&pert=15> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=499)
2021-04-09 16:36:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=499&pert=18> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=499)
2021-04-09 16:36:10 [scrapy.core

2021-04-09 16:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=500&pert=9> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=500)
2021-04-09 16:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxii.diputados.gob.mx/votaciones_por_pernplxii.php?iddipt=500&pert=3> (referer: http://sitllxii.diputados.gob.mx/votaciones_diputados_xperiodonplxii.php?dipt=500)
2021-04-09 16:36:28 [scrapy.core.engine] INFO: Closing spider (finished)
2021-04-09 16:36:28 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 3069558,
 'downloader/request_count': 6793,
 'downloader/request_method_count/GET': 6793,
 'downloader/response_bytes': 202728954,
 'downloader/response_count': 6793,
 'downloader/response_status_count/200': 6793,
 'elapsed_time_seconds': 2603.302277,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 4, 9, 21, 36, 28, 729

In [8]:
list_of_dipus = [value for value in dipus.values()]

In [9]:
list_of_votos = [value for value in por_votaciones.values()]

In [10]:
with open('diputados_62.json', 'a') as f:
            json.dump(list_of_dipus, f, indent=4, ensure_ascii=False) # antes dipus

In [11]:
with open('votaciones_62.json', 'a') as f:
            json.dump(list_of_votos, f, indent=4, ensure_ascii=False)

In [12]:
nom = ''
limpiaText(nom)

''

In [10]:
creaCurp('J. Pilar Moreno Montoya','')

'J.PILARMORENOMONTOYA'